In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error

In [2]:
train=pd.read_csv('../input/scrabble-player-rating/train.csv').fillna(-1)
print(train.head())
print(train.dtypes)

   game_id        nickname  score  rating
0        1       BetterBot    335    1637
1        1           stevy    429    1500
2        3       davidavid    440    1811
3        3       BetterBot    318    2071
4        4  Inandoutworker    119    1473
game_id      int64
nickname    object
score        int64
rating       int64
dtype: object


In [3]:
print(train.describe())

             game_id          score         rating
count  100820.000000  100820.000000  100820.000000
mean    36367.328605     399.029022    1875.592938
std     21020.288705      75.184778     231.791016
min         1.000000     -64.000000    1033.000000
25%     18139.000000     352.000000    1664.000000
50%     36339.500000     398.000000    1907.000000
75%     54625.000000     445.000000    2060.000000
max     72773.000000    1132.000000    2510.000000


In [4]:
print(train.isnull().sum())

game_id     0
nickname    0
score       0
rating      0
dtype: int64


In [5]:
print(train.shape)

(100820, 4)


In [6]:
print(train.isnull().sum())

game_id     0
nickname    0
score       0
rating      0
dtype: int64


In [7]:
train.nickname=pd.to_numeric(train.nickname,errors='coerce')

In [8]:
#train_ohe=pd.get_dummies(train)

In [9]:
#print(train_ohe.head(15))

In [10]:
#print(train_ohe.shape)

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
regr = RandomForestRegressor(max_depth=8, random_state=42,n_estimators=200)
X= train.drop('rating',axis=1).fillna(-1)
y=train.rating
train_X,test_X,train_y,test_y = train_test_split(X,y,test_size=0.20,random_state=42)
regr.fit(train_X,train_y)


RandomForestRegressor(max_depth=8, n_estimators=200, random_state=42)

In [12]:
pred= regr.predict(test_X)
print("MAE:",np.sqrt(mean_squared_error(pred,test_y)))

MAE: 208.97557790296975


In [13]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=4)
train=train.reset_index()
scores=[]
for train_idx,test_idx in kf.split(train):
    train_X=train.iloc[train_idx].drop('rating',axis=1).fillna(-1)
    test_X= train.iloc[test_idx].drop('rating',axis=1).fillna(-1)
    train_y=train.iloc[train_idx].rating
    test_y=train.iloc[test_idx].rating
    regr1 = RandomForestRegressor(max_depth=8, random_state=42,n_estimators=400)
    regr1.fit(train_X,train_y)
    pred= regr1.predict(test_X)
    msqe=np.sqrt(mean_squared_error(test_y,pred))
    scores.append(msqe)
    
    

In [14]:
print(scores)

[215.36656851953424, 209.32735401598066, 210.309929949046, 224.7422999162212]


In [15]:
check=pd.read_csv('../input/scrabble-player-rating/test.csv').fillna(-1)
print(check.head())

   game_id   nickname  score  rating
0        2      Super    488    -1.0
1        2  BetterBot    401  2000.0
2        7    STEEBot    377  2082.0
3        7     Priya1    379    -1.0
4       11    STEEBot    334  1829.0


In [16]:
print(check)

       game_id    nickname  score  rating
0            2       Super    488    -1.0
1            2   BetterBot    401  2000.0
2            7     STEEBot    377  2082.0
3            7      Priya1    379    -1.0
4           11     STEEBot    334  1829.0
...        ...         ...    ...     ...
44721    72762   kyjo55555    367    -1.0
44722    72768    HastyBot    524  2356.0
44723    72768  Maximilian    357    -1.0
44724    72769     STEEBot    626  2110.0
44725    72769    browndav    384    -1.0

[44726 rows x 4 columns]


In [17]:
print(check.dtypes)

game_id       int64
nickname     object
score         int64
rating      float64
dtype: object


In [18]:
X_check=check[~(check['nickname'].str.contains("Bot"))]

In [19]:
print(X_check.shape)

(22363, 4)


In [20]:
print(X_check.isnull().sum())

game_id     0
nickname    0
score       0
rating      0
dtype: int64


In [21]:
print(X_check.head(15))

    game_id       nickname  score  rating
0         2          Super    488    -1.0
3         7         Priya1    379    -1.0
5        11     TileRunner    462    -1.0
7        14     Anfield223    359    -1.0
8        27     friesbasil    456    -1.0
10       28       Eloquent    482    -1.0
13       32          Super    420    -1.0
14       40       eugenius    399    -1.0
16       41         marvin    295    -1.0
19       44       jlevedit    288    -1.0
20       55          urppp    448    -1.0
23       57          Jaish    296    -1.0
24       60   hannahbanana    373    -1.0
26       61  peperomartini    454    -1.0
28       62         Menonr    384    -1.0


In [22]:
X_check=X_check.drop('rating',axis=1)

In [23]:
X_check.nickname=pd.to_numeric(X_check.nickname,errors='coerce').fillna(-1)

In [24]:
X_check.dtypes

game_id       int64
nickname    float64
score         int64
dtype: object

In [25]:
#X_check_ohe=pd.get_dummies(X_check)

In [26]:
predict= regr.predict(X_check)

In [27]:
output=pd.DataFrame({'game_id':X_check.game_id,'rating':predict.astype(int)})

In [28]:
print(output.head())

   game_id  rating
0        2    1936
3        7    1929
5       11    1928
7       14    1766
8       27    1998


In [29]:
output.to_csv('submission.csv',index=False)